# Value at Risk (VaR) Methods — Overview

**Value at Risk (VaR)** is a statistical measure that quantifies the potential loss in value of a portfolio over a defined period for a given confidence level.

> "We are X% confident that we will not lose more than $Y in the next N days."

---

## Three Main VaR Approaches

| Method | Pros | Cons |
|--------|------|------|
| **Historical Simulation** | No distribution assumptions, captures fat tails | Past may not predict future, needs lots of data |
| **Parametric (Variance-Covariance)** | Fast, analytically tractable | Assumes normality, underestimates tail risk |
| **Monte Carlo Simulation** | Flexible, handles complex instruments | Computationally expensive, model-dependent |

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats

np.random.seed(42)

# Generate synthetic portfolio returns (2 years of daily data)
n_days = 504
returns = np.random.normal(0.0005, 0.015, n_days)  # ~24% annual vol
# Add some fat tails (realistic market behavior)
returns[np.random.choice(n_days, 20, replace=False)] += np.random.choice([-1, 1], 20) * np.random.uniform(0.03, 0.06, 20)

portfolio_value = 1_000_000  # $1M portfolio

print(f"Sample Size: {n_days} days")
print(f"Mean Daily Return: {np.mean(returns)*100:.3f}%")
print(f"Daily Volatility: {np.std(returns)*100:.3f}%")
print(f"Annualized Vol: {np.std(returns)*np.sqrt(252)*100:.1f}%")

## 1. Historical Simulation VaR

The simplest approach: use the actual historical return distribution.

$$VaR_{\alpha} = -\text{Percentile}(R, 1-\alpha) \times \text{Portfolio Value}$$

**Steps:**
1. Collect historical returns
2. Sort returns from worst to best
3. Find the return at the (1-α) percentile
4. Scale by portfolio value

In [ ]:
def historical_var(returns, confidence_level=0.95):
    """Calculate VaR using historical simulation."""
    var_pct = np.percentile(returns, (1 - confidence_level) * 100)
    return -var_pct  # Return as positive number (loss)

def historical_cvar(returns, confidence_level=0.95):
    """Calculate Expected Shortfall (CVaR) - average loss beyond VaR."""
    var_pct = np.percentile(returns, (1 - confidence_level) * 100)
    cvar_pct = returns[returns <= var_pct].mean()
    return -cvar_pct

# Calculate Historical VaR at different confidence levels
for conf in [0.90, 0.95, 0.99]:
    var = historical_var(returns, conf)
    cvar = historical_cvar(returns, conf)
    print(f"{conf*100:.0f}% VaR:  {var*100:.2f}% (${var * portfolio_value:,.0f})")
    print(f"{conf*100:.0f}% CVaR: {cvar*100:.2f}% (${cvar * portfolio_value:,.0f})")
    print()

## 2. Parametric (Variance-Covariance) VaR

Assumes returns are normally distributed.

$$VaR_{\alpha} = -(\mu + z_{\alpha} \cdot \sigma) \times \text{Portfolio Value}$$

Where:
- $\mu$ = mean return
- $\sigma$ = standard deviation of returns
- $z_{\alpha}$ = z-score at confidence level (e.g., -1.645 for 95%)

**For multi-day VaR** (assuming i.i.d. returns):
$$VaR_{T} = VaR_1 \times \sqrt{T}$$

In [ ]:
def parametric_var(returns, confidence_level=0.95, holding_period=1):
    """Calculate VaR using parametric (normal) method."""
    mu = np.mean(returns)
    sigma = np.std(returns)
    z_score = stats.norm.ppf(1 - confidence_level)
    var_1d = -(mu + z_score * sigma)
    return var_1d * np.sqrt(holding_period)

def parametric_cvar(returns, confidence_level=0.95, holding_period=1):
    """Calculate parametric CVaR (Expected Shortfall)."""
    mu = np.mean(returns)
    sigma = np.std(returns)
    z_score = stats.norm.ppf(1 - confidence_level)
    # ES for normal distribution
    es = mu - sigma * stats.norm.pdf(z_score) / (1 - confidence_level)
    return -es * np.sqrt(holding_period)

print("Parametric VaR (Normal Distribution Assumption)")
print("="*50)
for conf in [0.90, 0.95, 0.99]:
    var_1d = parametric_var(returns, conf, 1)
    var_10d = parametric_var(returns, conf, 10)
    print(f"{conf*100:.0f}% 1-day VaR:  {var_1d*100:.2f}% (${var_1d * portfolio_value:,.0f})")
    print(f"{conf*100:.0f}% 10-day VaR: {var_10d*100:.2f}% (${var_10d * portfolio_value:,.0f})")
    print()

## 3. Monte Carlo Simulation VaR

Generate thousands of possible future scenarios using a stochastic model.

**Common Models:**
- **Geometric Brownian Motion (GBM)**: $dS = \mu S dt + \sigma S dW$
- **GARCH**: time-varying volatility
- **Jump-diffusion**: captures sudden market moves

**Steps:**
1. Estimate model parameters from historical data
2. Simulate many price paths
3. Calculate P&L for each scenario
4. Find the VaR percentile from simulated distribution

In [ ]:
def monte_carlo_var(mu, sigma, portfolio_value, n_simulations=100_000, 
                    confidence_level=0.95, holding_period=1):
    """Calculate VaR using Monte Carlo simulation (GBM model)."""
    # Simulate returns over holding period
    dt = holding_period
    random_returns = np.random.normal(
        (mu - 0.5 * sigma**2) * dt,
        sigma * np.sqrt(dt),
        n_simulations
    )
    
    # Calculate portfolio changes
    portfolio_returns = np.exp(random_returns) - 1
    portfolio_pnl = portfolio_value * portfolio_returns
    
    # VaR and CVaR
    var = -np.percentile(portfolio_pnl, (1 - confidence_level) * 100)
    cvar = -portfolio_pnl[portfolio_pnl <= -var].mean() if np.any(portfolio_pnl <= -var) else var
    
    return var, cvar, portfolio_pnl

# Estimate parameters from historical data
mu_annual = np.mean(returns) * 252
sigma_annual = np.std(returns) * np.sqrt(252)

print(f"Estimated Annual Drift: {mu_annual*100:.2f}%")
print(f"Estimated Annual Vol: {sigma_annual*100:.2f}%")
print()

# Monte Carlo VaR
for conf in [0.95, 0.99]:
    var_1d, cvar_1d, pnl = monte_carlo_var(
        mu_annual/252, sigma_annual/np.sqrt(252), 
        portfolio_value, confidence_level=conf, holding_period=1
    )
    var_10d, cvar_10d, _ = monte_carlo_var(
        mu_annual/252, sigma_annual/np.sqrt(252), 
        portfolio_value, confidence_level=conf, holding_period=10
    )
    print(f"Monte Carlo {conf*100:.0f}% 1-day VaR:  ${var_1d:,.0f} (CVaR: ${cvar_1d:,.0f})")
    print(f"Monte Carlo {conf*100:.0f}% 10-day VaR: ${var_10d:,.0f} (CVaR: ${cvar_10d:,.0f})")
    print()

In [ ]:
# Visualize Monte Carlo simulation
_, _, pnl_sim = monte_carlo_var(
    mu_annual/252, sigma_annual/np.sqrt(252), 
    portfolio_value, n_simulations=50_000, confidence_level=0.95
)

var_95 = -np.percentile(pnl_sim, 5)
cvar_95 = -pnl_sim[pnl_sim <= -var_95].mean()

fig = go.Figure()
fig.add_trace(go.Histogram(x=pnl_sim, nbinsx=100, name='P&L Distribution'))
fig.add_vline(x=-var_95, line_dash='dash', line_color='red', 
              annotation_text=f'95% VaR: -${var_95:,.0f}')
fig.add_vline(x=-cvar_95, line_dash='dash', line_color='darkred',
              annotation_text=f'95% CVaR: -${cvar_95:,.0f}')
fig.add_vrect(x0=pnl_sim.min(), x1=-var_95, fillcolor='red', opacity=0.2,
              annotation_text='Tail Risk Zone')

fig.update_layout(
    title='Monte Carlo VaR: 1-Day P&L Distribution (50,000 simulations)',
    xaxis_title='Portfolio P&L ($)',
    yaxis_title='Frequency',
    template='plotly_white'
)
fig

## Method Comparison

Let's compare all three methods side by side.

In [ ]:
# Compare all methods
conf = 0.95

hist_var = historical_var(returns, conf) * portfolio_value
hist_cvar = historical_cvar(returns, conf) * portfolio_value

param_var = parametric_var(returns, conf) * portfolio_value
param_cvar = parametric_cvar(returns, conf) * portfolio_value

mc_var, mc_cvar, _ = monte_carlo_var(
    mu_annual/252, sigma_annual/np.sqrt(252), 
    portfolio_value, confidence_level=conf
)

comparison = pd.DataFrame({
    'Method': ['Historical', 'Parametric', 'Monte Carlo'],
    '95% VaR ($)': [hist_var, param_var, mc_var],
    '95% CVaR ($)': [hist_cvar, param_cvar, mc_cvar]
})
comparison['VaR (%)'] = comparison['95% VaR ($)'] / portfolio_value * 100
comparison['CVaR (%)'] = comparison['95% CVaR ($)'] / portfolio_value * 100

print("VaR Method Comparison ($1M Portfolio, 1-day, 95% Confidence)")
print("="*65)
print(comparison.to_string(index=False))

In [ ]:
# Visualization: Historical vs Normal distribution
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Return Distribution: Historical vs Normal', 'Q-Q Plot (Normality Check)'
))

# Histogram comparison
x_range = np.linspace(returns.min(), returns.max(), 100)
normal_pdf = stats.norm.pdf(x_range, np.mean(returns), np.std(returns))

fig.add_trace(go.Histogram(x=returns, nbinsx=50, histnorm='probability density', 
                           name='Historical', opacity=0.7), row=1, col=1)
fig.add_trace(go.Scatter(x=x_range, y=normal_pdf, mode='lines', 
                         name='Normal Fit', line=dict(color='red', width=2)), row=1, col=1)

# Q-Q plot
sorted_returns = np.sort(returns)
theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(returns)))
theoretical_quantiles = theoretical_quantiles * np.std(returns) + np.mean(returns)

fig.add_trace(go.Scatter(x=theoretical_quantiles, y=sorted_returns, 
                         mode='markers', name='Data', marker=dict(size=3)), row=1, col=2)
fig.add_trace(go.Scatter(x=[returns.min(), returns.max()], 
                         y=[returns.min(), returns.max()],
                         mode='lines', name='45° line', 
                         line=dict(dash='dash', color='red')), row=1, col=2)

fig.update_layout(template='plotly_white', showlegend=True)
fig.update_xaxes(title_text='Return', row=1, col=1)
fig.update_xaxes(title_text='Theoretical Quantiles', row=1, col=2)
fig.update_yaxes(title_text='Density', row=1, col=1)
fig.update_yaxes(title_text='Sample Quantiles', row=1, col=2)
fig.show()

# Normality tests
jb_stat, jb_p = stats.jarque_bera(returns)
print(f"Jarque-Bera test: statistic={jb_stat:.2f}, p-value={jb_p:.4f}")
print(f"Skewness: {stats.skew(returns):.3f}")
print(f"Excess Kurtosis: {stats.kurtosis(returns):.3f}")
print("\nNote: Fat tails (excess kurtosis > 0) mean parametric VaR may underestimate risk.")

## CVaR (Expected Shortfall)

**Conditional VaR (CVaR)**, also called **Expected Shortfall (ES)**, answers:

> "If we do lose more than VaR, how much do we expect to lose on average?"

$$CVaR_{\alpha} = E[L \mid L > VaR_{\alpha}]$$

**Advantages over VaR:**
- Coherent risk measure (satisfies subadditivity)
- Captures tail severity, not just tail probability
- Required by Basel III for market risk

---

## Quick Reference

| Aspect | Historical | Parametric | Monte Carlo |
|--------|------------|------------|-------------|
| **Data needs** | Long history | Short history | Model parameters |
| **Assumptions** | Past = Future | Normality | Chosen model |
| **Fat tails** | Captured | Underestimated | Model-dependent |
| **Speed** | Fast | Very fast | Slow |
| **Complex portfolios** | Difficult | Limited | Flexible |

### Basel III/IV Requirements
- 10-day holding period
- 99% confidence level
- Use Expected Shortfall instead of VaR
- Stressed VaR based on crisis period data